# Libraries

In [1]:
import random
import math
from collections import defaultdict

# Reading File

In [2]:

def read_dataset(file_path):
    dataset = defaultdict(list)     # a dictionary where each key is associated with a list by default.
                                    # Key = index, Value = List
    
    with open(file_path, 'r') as file:
        for line_num, line in enumerate(file, 1):
            line = line.strip()
           
            # Split the line into parts based on the expected format
            parts = line.split(',')
            
            # The first part is the vertex
            vertex = int(parts[0])
            
            # The last part is the cost
            cost = float(parts[-1])
            
            # Everything in between is the parent set
            parent_set_str = ','.join(parts[1:-1])
            parent_set = tuple(map(int, parent_set_str.strip('{}').split(','))) if parent_set_str.strip('{}') else ()
            
            dataset[vertex].append((parent_set, cost))
           
    
    return dataset

# Computing the Cost

In [3]:

def compute_cost(ordering, dataset):
    total_cost = 0
    
    # Loop through each vertex in the ordering
    for i, vertex in enumerate(ordering):
        min_cost = 100000
        
        # Get all parent sets for this vertex
        for parent_set, cost in dataset[vertex]:
            
            # Check if all parents come before the vertex in the ordering
            is_consistent = True
            for parent in parent_set:
                if parent not in ordering[:i]:
                    is_consistent = False
                    break
            
            # If consistent, update the minimum cost
            if is_consistent:
                min_cost = min(min_cost, cost)
        
        # Add the minimum cost to the total
        if min_cost != 100000:
            total_cost += min_cost
        else:
            # If no consistent parent set, this is not a valid ordering
            return 100000
    
    return total_cost


# Hill Climber

In [4]:

def hill_climbing(dataset, max_iterations=1000):
    vertices = list(dataset.keys())

    
    # Start with a random ordering
    current_order = list(vertices)
    random.shuffle(current_order)
    current_cost = compute_cost(current_order, dataset)
    
    for _ in range(max_iterations):
        improved = False
        
        # Try swapping each pair of vertices
        for i in range(len(vertices)):
            for j in range(i + 1, len(vertices)):
                
                # Create a new ordering by swapping two vertices
                new_order = current_order.copy()
                new_order[i], new_order[j] = new_order[j], new_order[i]
                new_cost = compute_cost(new_order, dataset)
                
                # If better, keep it
                if new_cost < current_cost:
                    current_order, current_cost = new_order, new_cost
                    improved = True
                    break
            if improved:
                break
        
        # If no improvement was found, we're at a local optimum
        if not improved:
            break
    
    return current_order, current_cost

# Simulated Annealing

In [5]:

def simulated_annealing(dataset, max_iterations=1000, decay_factor=0.99, initial_probability=1):
    vertices = list(dataset.keys())
    
    # Start with a random ordering
    current_order = list(vertices)
    random.shuffle(current_order)
    current_cost = compute_cost(current_order, dataset)
    
    best_order = current_order.copy()
    best_cost = current_cost
    
    probability = initial_probability
    
    for _ in range(max_iterations):
        
        # Generate a neighbor by swapping two random vertices
        i, j = random.sample(range(len(vertices)), 2)
        new_order = current_order.copy()
        new_order[i], new_order[j] = new_order[j], new_order[i]
        new_cost = compute_cost(new_order, dataset)
        
        # Decide whether to accept the new solution
        if new_cost < current_cost or random.random() < probability:
            current_order, current_cost = new_order, new_cost
            
            # Update best solution if better
            if current_cost < best_cost:
                best_order, best_cost = current_order.copy(), current_cost
        
        # Reduce the probability of accepting worse solutions over time
        probability *= decay_factor
        
        # Stop if the probability becomes too low
        if probability < 0.01:
            break
    
    return best_order, best_cost


# Greedy Best First Search

In [6]:

def greedy_search(dataset):
    vertices = list(dataset.keys())
    ordering = []
    remaining_vertices = set(vertices)
    
    while remaining_vertices:
        best_vertex = None
        best_cost = 100000
        
        # Find the best vertex to add next
        for vertex in remaining_vertices:
            min_cost = 100000
            for parent_set, cost in dataset[vertex]:
                
                # Check if all parents are already in our ordering
                if all(parent in ordering for parent in parent_set):
                    min_cost = min(min_cost, cost)
            
            if min_cost < best_cost:
                best_cost = min_cost
                best_vertex = vertex
        
        # Add the best vertex to our ordering
        ordering.append(best_vertex)
        remaining_vertices.remove(best_vertex)
    
    final_cost = compute_cost(ordering, dataset)
    return ordering, final_cost

# Comparing All the Algo's

In [7]:
# Run all algorithms and find the best result

                        # File       # File Name
def compare_algorithms(dataset_path, dataset_name):
    print(f"\nAnalyzing {dataset_name}")
    print("-" * 40)
    
    # Load dataset
    dataset = read_dataset(dataset_path)
    print(f"Dataset has {len(dataset)} vertices")
    
    # Run each algorithm
    greedy_order, greedy_cost = greedy_search(dataset)
    print(f"1. Greedy Search: Cost = {greedy_cost}")
    
    hc_order, hc_cost = hill_climbing(dataset)
    print(f"2. Hill Climbing: Cost = {hc_cost}")
    
    sa_order, sa_cost = simulated_annealing(dataset)
    print(f"3. Simulated Annealing: Cost = {sa_cost}")
    
    # Find the best algorithm
    if greedy_cost <= hc_cost and greedy_cost <= sa_cost:
        best_algo = "Greedy Search"
        best_order = greedy_order
        best_cost = greedy_cost
        
    elif hc_cost <= greedy_cost and hc_cost <= sa_cost:
        best_algo = "Hill Climbing"
        best_order = hc_order
        best_cost = hc_cost
        
    else:
        best_algo = "Simulated Annealing"
        best_order = sa_order
        best_cost = sa_cost
    
    print(f"\nBest result: {best_algo} with cost {best_cost}")
    print(f"Best ordering: {best_order}")

# Start

In [8]:

# Define datasets
datasets = [
    ("data0.txt", "Dataset 1"),
    ("data1.txt", "Dataset 2"),
    ("data2.txt", "Dataset 3"),
    ("data3.txt", "Dataset 4")
]


# Sending to Compare each dataset
for file_path, name in datasets:
    compare_algorithms(file_path, name)


Analyzing Dataset 1
----------------------------------------
Dataset has 5 vertices
1. Greedy Search: Cost = 465.43499999999995
2. Hill Climbing: Cost = 465.43399999999997
3. Simulated Annealing: Cost = 465.43399999999997

Best result: Hill Climbing with cost 465.43399999999997
Best ordering: [4, 2, 5, 3, 1]

Analyzing Dataset 2
----------------------------------------
Dataset has 18 vertices
1. Greedy Search: Cost = 3243.777
2. Hill Climbing: Cost = 3196.095
3. Simulated Annealing: Cost = 3194.8370000000004

Best result: Simulated Annealing with cost 3194.8370000000004
Best ordering: [1, 4, 5, 13, 11, 8, 6, 18, 12, 10, 3, 2, 7, 17, 16, 15, 9, 14]

Analyzing Dataset 3
----------------------------------------
Dataset has 19 vertices
1. Greedy Search: Cost = 1993.182
2. Hill Climbing: Cost = 1972.6349999999995
3. Simulated Annealing: Cost = 1975.2910000000002

Best result: Hill Climbing with cost 1972.6349999999995
Best ordering: [1, 8, 14, 10, 9, 11, 6, 13, 7, 3, 12, 19, 16, 17, 4, 5, 